In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#import tushare as ts
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io as scio
import time 

In [2]:
path_noisy = r'Input_Patches_3Dsyn2.csv'

In [3]:
data = pd.read_csv(path_noisy, header=None)

In [4]:
#type(data.shape[1])

In [5]:
data = data.T

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727
0,-0.222543,0.189528,-0.079685,-0.601468,-0.364523,-0.116314,-0.382804,-0.024234,0.120265,0.200890,...,-0.089436,-0.206247,-0.012145,0.031979,-0.008258,-0.332086,-0.048137,0.399868,0.174985,-0.049027
1,0.043055,-0.293164,-0.321591,-0.148925,-0.438493,-0.063874,0.118441,-0.126278,0.362018,0.132566,...,-0.044108,-0.310033,0.427230,-1.286497,-0.276580,-0.520041,0.807820,-0.760938,1.760631,0.488653
2,-0.155733,0.207998,0.075974,-0.449269,-0.009123,0.061756,-0.365791,-0.067311,0.357381,0.515759,...,0.240191,-0.095355,-0.191619,0.100956,-0.262505,-0.355827,-0.109347,0.085700,-0.029677,0.012212
3,-0.009747,0.321426,-0.016235,0.021465,-0.120913,-0.430391,-0.346896,-0.171349,-0.349497,0.338090,...,-0.139184,-0.385162,0.037424,-0.015406,-0.112325,0.085214,-0.404483,-0.125265,-0.201255,0.015312
4,0.309502,-0.680192,-0.729620,-1.760106,0.020797,-0.483123,-0.136336,-0.446910,-0.499735,0.308189,...,-0.084996,0.448267,0.032877,-0.021542,-0.186922,0.152195,0.140575,-0.151105,-0.052745,0.099958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50710,0.242076,0.076601,0.092620,-0.163801,-0.139315,-0.047873,0.205326,-0.003199,-0.306427,-0.190762,...,0.419610,0.449289,-0.063851,-0.005148,-0.048765,-0.149019,0.286533,-0.018275,-0.037540,0.392779
50711,0.451109,0.364099,0.227085,0.243715,-0.304514,-0.040334,-0.073196,-0.164095,-0.250486,0.345374,...,-0.163986,0.221959,0.073660,0.022314,-0.025043,0.128843,-0.216760,0.502266,0.007384,-0.327360
50712,0.332402,0.736117,0.668807,0.784152,-0.291261,-0.218808,0.105562,-0.128218,0.118664,-0.265187,...,-0.188547,-0.446252,-0.033322,0.010333,-0.232097,-0.271521,-0.093345,-0.010404,0.017806,-0.623515
50713,-0.088742,0.156048,0.617304,0.725424,0.158205,-0.013956,-0.080057,-0.008036,-0.140840,-0.069499,...,-0.359485,-0.359113,0.288803,0.014931,0.123114,0.353993,0.144014,-0.107187,-0.246051,0.090739


In [7]:
data = data.astype(np.float32)
data= torch.from_numpy(data.values)

In [8]:
data.shape

torch.Size([50715, 1728])

In [9]:
#将前80%作为训练集，后20%作为测试集
train_size = int(len(data) * 0.8)
train = data[:train_size]
vaild = data[train_size:]
print(train.shape)
print(vaild.shape)
batch_size1 = 64
w1 = 12
w2 = 12
w3 = 12

train_data= TensorDataset(train)
vaild_data= TensorDataset(vaild)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

vaild_loader = torch.utils.data.DataLoader(vaild_data,
                                          batch_size = 64,
                                          shuffle = True)

torch.Size([40572, 1728])
torch.Size([10143, 1728])


In [10]:
#定义Fully connected (FC) block
class FCB(nn.Module):
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        self.activation = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x) 
        x = self.bn(x)
        x = self.dropout(x)
        
        return x

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, dropout=0.1):
        super().__init__()


        self.fcb0 = FCB(input_size, 128, dropout)
        self.fcb1 = FCB(128, 64, dropout)

        self.fcb2 = FCB(64, 32, dropout)

        self.fcb3 = FCB(32, 16, dropout)

        self.fcb4 = FCB(16, 8, dropout)

        self.fcb5 = FCB(8, 4, dropout)

        self.fcb6 = FCB(4, 4, dropout)

        


    def forward(self, x):

        #x = x.reshape(x.shape[0],1,x.shape[1])

        x1 = self.fcb0(x)#x->128
        x2 = self.fcb1(x1)

        x3 = self.fcb2(x2)

        x4 = self.fcb3(x3)

        x5 = self.fcb4(x4)

        x6 = self.fcb5(x5)

        x7 = self.fcb6(x6)

        x8 = self.fcb6(x7)
        

        

        return x2,x3,x4,x5,x6,x7,x8

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_size, dropout=0.1):
        super().__init__()
        self.pab1 = FCB(8, 8, dropout)
        self.pab2 = FCB(16, 16, dropout)
        self.pab3 = FCB(32, 32, dropout)
        self.pab4 = FCB(64, 64, dropout)
        self.pab5 = FCB(128, 128, dropout)
        self.pab6 = FCB(128, output_size, dropout)

        self.activation = nn.Identity()


    def forward(self,x2, x3,x4,x5,x6,x7,x8):
        
        x = torch.cat((x6,x8),dim=1)
        x = self.pab1(x)
        x = torch.cat((x,x5),dim=1)
        x = self.pab2(x)
        x = torch.cat((x,x4),dim=1)
        x = self.pab3(x)
        x = torch.cat((x,x3),dim=1)
        x = self.pab4(x)
        x = torch.cat((x,x2),dim=1)
        x = self.pab5(x)
        x = self.pab6(x)

        x = self.activation(x)
        return x

In [13]:
class AutoEncoder(nn.Module):
    def __init__(self,input_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(output_size)

    def forward(self, x):
        x2, x3,x4,x5,x6,x7,x8= self.encoder(x)
        x = self.decoder(x2, x3,x4,x5,x6,x7,x8)

        return x

In [14]:
device = torch.device("cuda") 
model = AutoEncoder(w1*w2*w3,w1*w2*w3).to(device)
#将模型转移到GPU上
#criterion = MeanHuberLoss(delta=1.3)
#criterion = WelschLoss(delta=0.5)
#criterion = Loss0(delta=0.46,r=0.05)#0.5 and 0.2,SNR:-8dB ok
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [15]:
start_time = time.time()
es_cnt = 0
es_thres = 5
prev_train_loss = float('inf')
loss_train = []
loss_vaild = []
num_epochs = 100 # 总训练轮数
#num_batch_train = 0
for epoch in range(num_epochs):
  #train_bar = tqdm(train_loader)
  train_loss = 0.0
  
  for i , (batch) in enumerate(train_loader):

    # 数据转到device
    train_batch = batch[0].to(device)
    
    # 训练步骤  
    optimizer.zero_grad()
    outputs = model(train_batch)
    loss = criterion(outputs, train_batch)
    loss.backward() 
    optimizer.step()
    
    train_loss += loss.item()
    #num_batch_train +=1
  #train_loss除以所有bacth个数
  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
  loss_train.append(train_loss)
    



  # 验证
  valid_loss = 0.0
  #num_batch_vaild = 0
  with torch.no_grad():
    for i , (batch) in enumerate(vaild_loader):
    #for batch in vaild_loader:
    
      val_batch = batch[0].to(device)
      
      outputs = model(val_batch)
      loss = criterion(outputs, val_batch)
      valid_loss += loss.item()
      #num_batch_vaild += 1
    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
    loss_vaild.append(valid_loss)
    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))

    
    # Early stopping
    if train_loss - prev_train_loss >= 0:
        es_cnt += 1
    else:
        #es_cnt = 0
        pass

    if es_cnt >= es_thres:
        print(f"Early stopped at epoch {epoch}, train loss stop improving")
        break  

    prev_train_loss = train_loss
  print('loss_train: ', loss_train)
  print('vaild_train: ',loss_vaild)          
print("Training finished")
current_time = time.time()
time_sum = current_time-start_time
print(time_sum)

Epoch [1/100], Train Loss: 0.5753, Valid Loss: 0.2348
loss_train:  [0.5752540377882377]
vaild_train:  [0.23476196450632322]
Epoch [2/100], Train Loss: 0.1475, Valid Loss: 0.0927
loss_train:  [0.5752540377882377, 0.14746559881355484]
vaild_train:  [0.23476196450632322, 0.0926899606326841]
Epoch [3/100], Train Loss: 0.0862, Valid Loss: 0.0831
loss_train:  [0.5752540377882377, 0.14746559881355484, 0.08618732833241814]
vaild_train:  [0.23476196450632322, 0.0926899606326841, 0.08313200243239133]
Epoch [4/100], Train Loss: 0.0822, Valid Loss: 0.0829
loss_train:  [0.5752540377882377, 0.14746559881355484, 0.08618732833241814, 0.08220572183231827]
vaild_train:  [0.23476196450632322, 0.0926899606326841, 0.08313200243239133, 0.08290299660754653]
Epoch [5/100], Train Loss: 0.0816, Valid Loss: 0.0825
loss_train:  [0.5752540377882377, 0.14746559881355484, 0.08618732833241814, 0.08220572183231827, 0.08163053188649262]
vaild_train:  [0.23476196450632322, 0.0926899606326841, 0.08313200243239133, 0.0829

In [16]:
loss_train = pd.DataFrame(loss_train)
loss_vaild = pd.DataFrame(loss_vaild)

loss = pd.concat([loss_train,loss_vaild],axis=1)

In [17]:
loss.columns = ['train_loss','vaild_loss']

In [18]:
torch.save(model.state_dict(), r'.\model_3dsyn2patch.pth')

In [20]:
model = AutoEncoder(w1*w2*w3,w1*w2*w3).to(device)
data = data.to(device)
model.load_state_dict(torch.load(r'.\model_3dsyn2patch.pth'))
model.eval()
with torch.no_grad():
    output = model(data)
    #loss = criterion(output, data)
print(output.shape)

torch.Size([50715, 1728])


In [22]:
output = output.cpu()
output = output.numpy()
output = pd.DataFrame(output)

In [23]:
loss.to_csv(r'loss_3dsyn2patch.csv',index=False)

In [24]:
output.to_csv(r'./output_3dsyn2patch.csv',index=None,header=None)

In [ ]:
#loss_0 = []
#start_time = time.time()
#es_cnt = 0
#es_thres = 5
#prev_train_loss = float('inf')
#loss_train = []
#loss_vaild = []
#num_epochs = 100 # 总训练轮数
##num_batch_train = 0
#for epoch in range(num_epochs):
#  #train_bar = tqdm(train_loader)
#  train_loss = 0.0
#  
#  for i , (batch) in enumerate(train_loader):
#
#    # 数据转到device
#    train_batch = batch[0].to(device)
#    
#    # 训练步骤  
#    optimizer.zero_grad()
#    outputs = model(train_batch)
#    loss = criterion(outputs, train_batch)
#    loss.backward() 
#    optimizer.step()
#
#    loss_0.append(loss.item())
#
#    train_loss += loss.item()
#    #num_batch_train +=1
#  #train_loss除以所有bacth个数
#  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
#  loss_train.append(train_loss)
#    
#
#
#
#  # 验证
#  valid_loss = 0.0
#  #num_batch_vaild = 0
#  with torch.no_grad():
#    for i , (batch) in enumerate(vaild_loader):
#    #for batch in vaild_loader:
#    
#      val_batch = batch[0].to(device)
#      
#      outputs = model(val_batch)
#      loss = criterion(outputs, val_batch)
#      valid_loss += loss.item()
#      #num_batch_vaild += 1
#    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
#    loss_vaild.append(valid_loss)
#    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))
#
#    
#    # Early stopping
#    if train_loss - prev_train_loss >= 0:
#        es_cnt += 1
#    else:
#        #es_cnt = 0
#        pass
#
#    if es_cnt >= es_thres:
#        print(f"Early stopped at epoch {epoch}, train loss stop improving")
#        break  
#    
#
#
##   if epoch == 10:
##     torch.save(model.state_dict(), r'.\model_epochs10.pth')
##   elif epoch == 20:
##     torch.save(model.state_dict(), r'.\model_epochs20.pth')
##   elif epoch == 30:
##     torch.save(model.state_dict(), r'.\model_epochs30.pth')
##   elif epoch == 40:
##     torch.save(model.state_dict(), r'.\model_epochs40.pth')
##   elif epoch == 50:
##     torch.save(model.state_dict(), r'.\model_epochs50.pth')
##   elif epoch == 60:
##     torch.save(model.state_dict(), r'.\model_epochs60.pth')
##   elif epoch == 70:
##     torch.save(model.state_dict(), r'.\model_epochs70.pth')
##   elif epoch == 80:
##     torch.save(model.state_dict(), r'.\model_epochs80.pth')
##   elif epoch == 90:
##     torch.save(model.state_dict(), r'.\model_epochs90.pth')
##   elif epoch == 100:
##     torch.save(model.state_dict(), r'.\model_epochs100.pth')
##   else:
##       pass
#
#    prev_train_loss = train_loss
#  print('loss_train: ', loss_train)
#  print('vaild_train: ',loss_vaild)          
#print("Training finished")
#current_time = time.time()
#time_sum = current_time-start_time
#print(time_sum)